In [1]:
import sys
# win:
# project_path = 'D:/Code/repos/gad1b-redistribution/src'
# wsl:
project_path = '/mnt/d/Code/repos/pwreg'
sys.path.insert(1, f'{project_path}/pwreg')

from core.core import *
from visualize.viewers import *

import pickle
import pandas as pd

import inspect
# print(inspect.getsource(Something))

%load_ext autoreload
%autoreload 2

project_path

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
/home/ply/anaconda3/envs/ants/lib/python3.8/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


'/mnt/d/Code/repos/pwreg'

In [3]:
mask_tp1 = {'zmin': 51,'zmax': 168 + 1}
mask_tp2 = {'zmin': 51,'zmax': 164 + 1}

# make it a bit more constictive 
# mask_tp1 = {'zmin': 52,'zmax': 168}
# mask_tp2 = {'zmin': 52,'zmax': 164}

fixed = Image([0.658, 0.23, 0.23], filename = f'{project_path}/data/raw/img/no_change_control/Image_1-20FJ.ome.tif', mask = mask_tp1) # tp 1
moving = Image([0.658, 0.23, 0.23], filename = f'{project_path}/data/raw/img/no_change_control/Image_1-20FM.ome.tif', mask = mask_tp2) # tp 2

print(f'shape\nFixed {fixed.shape},\nMoving {moving.shape}')
print(f'masks :\nFixed {fixed.mask},\nMoving {moving.mask}')

shape
Fixed (118, 1024, 1024),
Moving (114, 1024, 1024)
masks :
Fixed {'zmin': 51, 'zmax': 169, 'xmin': None, 'xmax': None, 'ymin': None, 'ymax': None},
Moving {'zmin': 51, 'zmax': 165, 'xmin': None, 'xmax': None, 'ymin': None, 'ymax': None}


In [4]:
fixed_blc_size = list(fixed.img.shape)
moving_blc_size = list(moving.img.shape)
fixed_overlap = 0
moving_overlap = 0

bps = BlockPairs.from_image_pair(fixed, moving, fixed_blc_size, fixed_overlap, moving_blc_size, moving_overlap)
bps

1 blockpairs

In [6]:
bps.register(verbose = True)

  0%|                                                                                             | 0/1 [00:00<?, ?it/s]

fixed : ANTsImage (RAI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (118, 1024, 1024)
	 Spacing    : (0.658, 0.23, 0.23)
	 Origin     : (0.0, 0.0, 0.0)
	 Direction  : [1. 0. 0. 0. 1. 0. 0. 0. 1.]

moving : ANTsImage (RAI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (114, 1024, 1024)
	 Spacing    : (0.658, 0.23, 0.23)
	 Origin     : (0.0, 0.0, 0.0)
	 Direction  : [1. 0. 0. 0. 1. 0. 0. 0. 1.]



100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [04:26<00:00, 266.08s/it]


In [9]:
tag = 'v03312022_old_code'

Image([0.658, 0.23, 0.23], img = bps.pairs[0].warp()).imwrite(f'{project_path}/data/processed/registered/no_change_control/testing_synapses/1-20FM_to_1-20FJ_affine_{tag}.tif')
fixed.imwrite(f'{project_path}/data/processed/registered/no_change_control/testing_synapses/1-20FJ_cropped_{tag}.tif')

# bps.pairs[0].alignment['affine'].to_json(f'{project_path}/data/processed/registered/no_change_control/testing_synapses/1-20FJ_to_1-20FM_affine_global_alignment_{tag}.json')
# bps.to_json(f'{project_path}/data/processed/registered/no_change_control/testing_synapses/1-20FJ_to_1-20FM_affine_global_bps_{tag}.json')

# PW Reg

In [10]:
prereg = bps.pairs[0].warp(interpolator='linear')

In [11]:
fixed_blc_size = [18,102,102]
moving_blc_size = [18,102,102]
fixed_overlap = [2,5,5]
moving_overlap = [2,5,5]

bps = BlockPairs.from_image_pair(fixed, Image([0.658, 0.23, 0.23], img =prereg), fixed_blc_size, fixed_overlap, moving_blc_size, moving_overlap)
bps

/mnt/d/Code/repos/pwreg/pwreg/core/core.py:108: UserWarning:

Specified voxel size + overlap don't cover the whole image.Image size is [ 118 1024 1024], block size [ 18 102 102], overlap [2 5 5] results in [ 7.25       10.50515464 10.50515464] blocks.
Leaving some image out. 



700 blockpairs

In [12]:
bps.register(verbose = False)

100%|█████████████████████████████████████████████████████████████████████████████████| 700/700 [06:07<00:00,  1.90it/s]


In [13]:
tag = '18x102x102_2x5x5_old_code_v03312022'

bps.to_json(f'{project_path}/data/processed/registered/no_change_control/testing_synapses/1-20FJ_to_1-20FM_pw_block_pairs_{tag}.json')

padding = 10
fixed_blc = fixed.split(fixed_blc_size,fixed_overlap)
BlockView(fixed_blc, padding).write_volume(f'{project_path}/data/processed/registered/no_change_control/testing_synapses/blocks_1-20FJ_{tag}.tif')
BlockPairView(bps.pairs, padding).write_volume(f'{project_path}/data/processed/registered/no_change_control/testing_synapses/blocks_1-20FM_to_1-20FM_pwreg_{tag}.tif')

/mnt/d/Code/repos/pwreg/pwreg/core/core.py:108: UserWarning:

Specified voxel size + overlap don't cover the whole image.Image size is [ 118 1024 1024], block size [ 18 102 102], overlap [2 5 5] results in [ 7.25       10.50515464 10.50515464] blocks.
Leaving some image out. 



# Apply all transforms to points

In [14]:
segmentation_folder = '/mnt/d/Code/repos/psd95_segmentation/models/two_small_cnns/predictions/Gad1b_2021/Sequential_2tp_control/'
ml_seg1 = pd.read_csv(f'{segmentation_folder}ML_segmentation_1-20GA_12-22-2021_2CNN_gui.csv') # tp1
ml_seg2 = pd.read_csv(f'{segmentation_folder}ML_segmentation_1-20G6_12-22-2021_2CNN_gui.csv') # tp2

prob_thr = 0.5

ptc1 = Points(ml_seg1[['Z','Y','X']][ml_seg1["prob"]>prob_thr].to_numpy(), units='pix', resolution=[0.658, 0.23, 0.23])
ptc2 = Points(ml_seg2[['Z','Y','X']][ml_seg2["prob"]>prob_thr].to_numpy(), units='pix', resolution=[0.658, 0.23, 0.23])

ptc1 = ptc1.crop(mask_tp2, units='pix')
ptc2 = ptc2.crop(mask_tp1, units='pix')
ptc1 = ptc1.recenter([mask_tp1['zmin'],0,0], units = 'pix')
ptc2 = ptc2.recenter([mask_tp2['zmin'],0,0], units = 'pix')

ptc1.to_json(f'{project_path}/data/processed/registered/no_change_control/ptc1_global_{tag}.json')
ptc2.to_json(f'{project_path}/data/processed/registered/no_change_control/ptc2_global_{tag}.json')

af = AffineTransform.from_json(f'{project_path}/data/processed/registered/no_change_control/testing_synapses/1-20FM_to_1-20FJ_affine_{tag}.json')
ptc2_af = ptc2.transform(af, units='phs')
ptc2_af.to_json(f'{project_path}/data/processed/registered/no_change_control/testing_synapses/ptc2_1-20FM_to_1-20FJ_affine_{tag}.json')

ptc2_pw = ptc2_af.pw_transform(bps.pairs)
PointsList(ptc2_pw).to_json(f'{project_path}/data/processed/registered/no_change_control/testing_synapses/ptc2_1-20FM_to_1-20FJ_pwreg_{tag}.json')

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/d/Code/repos/pwreg/data/processed/registered/no_change_control/testing_synapses/1-20FM_to_1-20FJ_affine_18x102x102_2x5x5_old_code_v03312022.json'

# output stuff to be visualised 

In [25]:
fixed_blc = fixed.split(fixed_blc_size,fixed_overlap)
padding = 10

PointsBlockView(ptc1.split(fixed_blc), fixed_blc, padding).to_json(f'{project_path}/data/processed/registered/no_change_control/testing_synapses/ptc1_1-20FJ_{tag}.json')
PointsBlockView(ptc2_pw, fixed_blc, padding).to_json(f'{project_path}/data/processed/registered/no_change_control/testing_synapses/ptc2_1-20FM_to_1-20FJ_pwreg_{tag}.json')

## lost vs gained

In [32]:
from utils.synapse import *

In [43]:
radius = 1.0
synapse_study = PwSynapsePair(ptc1,ptc2_pw,radius)

100%|█████████████████████████████████████████████████████████████████████████████████| 700/700 [04:37<00:00,  2.53it/s]


In [45]:
PointsBlockView(synapse_study.lost, fixed_blc, padding).to_json(f'{project_path}/data/processed/registered/no_change_control/testing_synapses/ptc2_lost_1-20FM_to_1-20FJ_pwreg_v03242022.json')
PointsBlockView(synapse_study.gained, fixed_blc, padding).to_json(f'{project_path}/data/processed/registered/no_change_control/testing_synapses/ptc2_gained_1-20FM_to_1-20FJ_pwreg_v03242022.json')
PointsBlockView(synapse_study.uncB, fixed_blc, padding).to_json(f'{project_path}/data/processed/registered/no_change_control/testing_synapses/ptc2_uncB_1-20FM_to_1-20FJ_pwreg_v03242022.json')
PointsBlockView(synapse_study.uncA, fixed_blc, padding).to_json(f'{project_path}/data/processed/registered/no_change_control/testing_synapses/ptc2_uncA_1-20FM_to_1-20FJ_pwreg_v03242022.json')